In [1]:
import json
import requests
import pandas as pd

In [11]:
import io
import os

In [12]:
from google.cloud import vision
from google.cloud.vision import types
#imports the google cloud lib.

In [2]:
cx = 'insert_engine_ID'
key = 'insert_API_key'

In [20]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="insert_file_path_to_service_account_key"

In [3]:
# hq - appends logical AND to added search terms 
# hl - sets user interface language
# num - how many results to return
# sort - sort expression to apply to results
# start - index of first return

In [241]:
def look_up(yourQuestion): 
    url = 'https://www.googleapis.com/customsearch/v1' #need to attach this url so it can connect it with google API
    parameter = {'q':yourQuestion,'cx':cx,'key':key,} #dict of the question being asked, engine_ID, API_key(lets you use the API)
    result_page = requests.request('GET',url,params= parameter) #sends a HTTP request to get search results  
    results_json = json.loads(result_page.text) #loads the results into JSON format
    
    links = [item['link'] for item in results_json['items']] #list comprehension to turn all 'link' data into a list
    df = pd.DataFrame(links, columns=['link']) #makes a Dataframe with the columns mapped to the 'links' list
    df['title'] = [item['title'] for item in results_json['items']] #adds 'title' data to Dataframe
    df['htmlSnippet'] = [item['htmlSnippet'] for item in results_json['items']] #adds 'htmlSnippet' data to Dataframe
    
    next_index = results_json['queries']['nextPage'][0]['startIndex'] 
    #sets the index at 10 so we get another 10 results, the reason why we change have to do this is b/c this method only returns
    #10 results, there is another method below which returns any number of results 
 
    #---------------------------------------------------------------------------------------------------------------------------
    
    #code here is all the same as above but just with the data from the next 10 results(or data on the next page)
    para1 = {'q':yourQuestion,'cx':cx,'key':key,'start':next_index} 
    
    page1 = requests.request('GET',url,params= para1)
    results1 = json.loads(page1.text)
    
    links1 = [item['link'] for item in results1['items']]
    df1 = pd.DataFrame(links1, columns=['link'])
    df1['title'] = [item['title'] for item in results1['items']]
    df1['htmlSnippet'] = [item['htmlSnippet'] for item in results1['items']]
    
    data = pd.concat([df,df1],ignore_index=True) #merges the two dataframes 
    
    
    return data

In [242]:
#another way of getting search results
#source : https://stackoverflow.com/questions/41032472/how-to-query-an-advanced-search-with-google-customsearch-api

from googleapiclient.discovery import build
import pprint

my_api_key = key
my_cse_id = cx

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res['items']

results = google_search(
    'stackoverflow site:en.wikipedia.org', my_api_key, my_cse_id, num=10)

In [158]:
def detect_text(file_name):
    #finds text in file
    client = vision.ImageAnnotatorClient() #make a client
    
    with open(file_name,'rb') as image_file: #grabs the image givin the title
        content = image_file.read()
        
    image = types.Image(content = content) #loads image into memory
    
    response = client.text_detection(image = image)
    texts = response.text_annotations
    #prefroms text detection
    
    return texts[0].description.split('\n')
    #texts -> is returns a list, the frist index of the list contains
    #description, which can split be based on the \n, this yields 
    #the Question and the Answer choices 

In [180]:
def clean_up(text):
    text = [x.lower() for x in text] #lower-cases the whole list
    text = text[:-1] #gets rid of the last index, the last index contained '', which has no use
    
    
    for x in text: #this loop is for finding out which indices does the question takes up
        if '?' in x: #look for '?' in text[x]
            index = text.index(x) + 1 #once it finds the first instance of a '?' then it breaks the loop
            break
    #There could be a flaw here, b/c if the question had another ? in it, then that messes up the code
    
    text[0:index] = [' '.join(text[0:index])] #join the all the indices up until the one with a '?' in it

    return text

In [223]:
def turn_into_search_string(question):#has to be question with 3 multiple choices, in a list
    search_string = '{} AND ({} | {} | {})'.format(question[0],question[1],question[2],question[3])
    return search_string

In [209]:
#turns all the strings in the dataframe columns lowercase, making it lowercase helps with the search
def turn_into_lower_case(df): 
    df['title'] = df['title'].apply(lambda x: x.lower()) 
    df['htmlSnippet'] = df['htmlSnippet'].apply(lambda x: x.lower())

In [215]:
#counts up all the instances where it found the answer choices, works only for three choices
def count_answers(data,q):
    a1 = data['title'].apply(lambda x: x.count(q[1])).sum() + data['htmlSnippet'].apply(lambda x: x.count(q[1])).sum()  
    a2 = data['title'].apply(lambda x: x.count(q[2])).sum() + data['htmlSnippet'].apply(lambda x: x.count(q[2])).sum()
    a3 = data['title'].apply(lambda x: x.count(q[3])).sum() + data['htmlSnippet'].apply(lambda x: x.count(q[3])).sum()
    return [a1,a2,a3]

18


In [283]:
#prints the question and frequency of the answer choices. Works only for three choices, so far. 
def print_count(count_list,q):
    print('{}\n{} : {} \n{} : {}\n{} : {}'.format(q[0],q[1],count_list[0],q[2],count_list[1],q[3],count_list[2]))

In [284]:
def get_answer():
    file_title = input() #asks for user input of the file
    file_name = file_title + '.PNG' #adds a png to the end
    question_list = detect_text(file_name) # finds text on it
    question_list = clean_up(question_list) #cleans up the text list returned
    searchable_string = turn_into_search_string(question_list) #turns question list into a search string
    df = look_up(searchable_string) #searchs web and collects the data
    turn_into_lower_case(df) #turns a part of the dataframe lowercase
    count_of_choices = count_answers(df,question_list) #counts up the choices found and compiles them into a list
    print_count(count_of_choices,question_list) #prints out the choice with the count number next to it
    

In [285]:
#same method as above but contains all the data, so user can mess around with it or something 

def get_answer_with_data():
    file_title = input() #asks for user input of the file(name of file in local folder) or you change to specify certain PATH
    file_name = file_title + '.PNG' #adds a png to the end, you can change to match whatever photo format you need
    question_list = detect_text(file_name) # finds text on it
    question_list = clean_up(question_list) #cleans up the text list returned
    searchable_string = turn_into_search_string(question_list) #turns question list into a search string
    df = look_up(searchable_string) #searchs web and collects the data
    turn_into_lower_case(df) #turns a part of the dataframe lowercase
    count_of_choices = count_answers(df,question_list) #counts up the choices found and compiles them into a list
    print_count(count_of_choices,question_list) #prints out the choice with the count number next to it
    return [file_name,question_list,searchable_string,df,count_of_choices]

In [286]:
get_answer() #correct answer is Busking

q2
what is the act of performing music in public places for tips?
busking : 26 
straw-capping : 0
panhandling : 3


In [287]:
get_answer() #correct answer is Snow Crash

q3
which novel popularized the term "avatar" in the current sense?
the player of games : 0 
snow crash : 15
neuromancer : 4


In [288]:
get_answer() #correct answer is the first one 
#as you can see, the vision API isn't a 100% correct. Notice 'these' is spelled 'tese'

q11
according to the u.s. government, which of tese is correct?
daylight saving time : 29 
daylight savings time : 7
daylight saving's time : 0


In [289]:
get_answer() #correct answer is Paris, there is still room for LOTS of improvement

q12
which of these cities has the biggest bike share program?
paris : 0 
amsterdam : 1
new york : 21


In [290]:
get_answer() #obs. correct answer is Pond, like I said room for LOTS of improvement

q1
which of these bodies of water is the smallest?
pond : 0 
sea : 33
ocean : 43


In [291]:
get_answer() #correct answer is Mrs.Doubtfire

q13
which of these robin williams movies is based on a novel?
mrs. doubtfire : 4 
dead poets society : 7
good morning, vietnam : 1


In [292]:
get_answer() #correct answer is birds' nests

q10
the asian delicacy known as bird's nest soup is made primarily from what?
birds' nests : 0 
fried noodles : 1
shredded cabbage : 2
